# Базы данных (Database) и СУБД

* Долговременное хранение данных
* Хранение связанных данных и отношенийй между ними, систематизация данных
* Эффективный поиск и селекция

## СУБД

* Доступ к данных из нескольких программ
* Обеспечение атомарности, согласованности, изолированности и долговечности данных (ACID)

## ACID
* Атомарность - гарантирует, что никакая транзакция не будет зафиксирована в системе частично.
* Согласованность - после заверешения транзакции данные целостны и не противоречат друг другу.
* Изолированность - во время выполнения транзакции параллельные транзакции не должны оказывать влияния на её результат.
* Долговечность - если транзакция успешно проведена, то значит данные действительно сохранены.

**SQL** - распространненный язык для составления запросов к БД

## Python Database API Specification v2.0

* [PEP 249](https://www.python.org/dev/peps/pep-0249/)
* Приведение модулей для работы с различным базами данных к единому интерфейсу.
* Обеспечивает универсальность работы и модульную замену
* [Python Database Propgramming](https://wiki.python.org/moin/DatabaseProgramming)

## SQLite

* Компактная, встраиваемая БД.
* Хранит данные в файле, блокирующая однопользовательская
* [Сайте](https://www.sqlite.org/index.html)
* [Wiki](https://ru.wikipedia.org/wiki/SQLite)

In [1]:
import sqlite3

Можно подключится к файлу БД

In [ ]:
# con = sqlite3.connect('example.db') 

Mожно создать БД в памяти, см. [In-memory database](https://en.wikipedia.org/wiki/In-memory_database)

In [2]:
con = sqlite3.connect(':memory:')

In [ ]:
# TODO(Придумать схему таблицы и код для генерации данных)
# TODO(Использование адаптеров и конверторов)

In [3]:
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE stocks
               (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
con.commit()


In [ ]:
# Successful, con.commit() is called automatically afterwards
with con:
    con.execute("", ("",))

In [ ]:
# Connection object used as context manager only commits or rollbacks transactions,
# so the connection object should be closed manually
con.close()

## SQAlchemy

* Высокоуровневая библиотека для работы с базами данными
* Для доступа в базу данных может использовать разные драйверы удовлетворяющий DPAPI

![](./sqla_engine_arch.png)

* **SQLAlchemy Core** --- Python DSL for SQL
* **SQLAlchemy ORM** --- object relational mapping

### Object Relational Mapping

Проблема:
 * Данные в БД хранятся в виде таблиц
 * Данные в программе представлены в виде классов и объектов
 * Возникает необходимость писать лишний код для взаимодейсвтия с базой данной --- CRUD операции: (Create, Read, Update, Delete)
Решение:
 * ORM технологии для автоматического отображения классов в базу данных

In [ ]:
import sqlalchemy
sqlalchemy.__version__

In [4]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

In [5]:
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String, DateTime
import datetime

metadata = MetaData()
table = Table('table', metadata,
    Column('id', Integer(), primary_key=True),
    Column('name', String(200), nullable=False),
    Column('timestamp', DateTime(), default=datetime.datetime.now),
)

metadata.create_all(engine)

2021-11-09 17:30:18,884 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 17:30:18,885 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("table")
2021-11-09 17:30:18,886 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-09 17:30:18,887 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("table")
2021-11-09 17:30:18,888 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-09 17:30:18,892 INFO sqlalchemy.engine.Engine 
CREATE TABLE "table" (
	id INTEGER NOT NULL, 
	name VARCHAR(200) NOT NULL, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2021-11-09 17:30:18,893 INFO sqlalchemy.engine.Engine [no key 0.00105s] ()
2021-11-09 17:30:18,894 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
stmt = table.insert().values(name="NAMENAME")
print(stmt)

INSERT INTO "table" (name, timestamp) VALUES (:name, :timestamp)


In [7]:
with engine.connect() as conn:
    r = conn.execute(stmt)
    print(r.inserted_primary_key)

2021-11-09 17:31:28,943 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 17:31:28,944 INFO sqlalchemy.engine.Engine INSERT INTO "table" (name, timestamp) VALUES (?, ?)
2021-11-09 17:31:28,946 INFO sqlalchemy.engine.Engine [generated in 0.00263s] ('NAMENAME', '2021-11-09 17:31:28.943561')
(1,)
2021-11-09 17:31:28,947 INFO sqlalchemy.engine.Engine ROLLBACK


In [8]:
metadata.drop_all(engine)

2021-11-09 17:32:23,135 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 17:32:23,137 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("table")
2021-11-09 17:32:23,137 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-09 17:32:23,138 INFO sqlalchemy.engine.Engine 
DROP TABLE "table"
2021-11-09 17:32:23,139 INFO sqlalchemy.engine.Engine [no key 0.00044s] ()
2021-11-09 17:32:23,140 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

class MySomeTable(Base):
    __tablename__ = "table"
    id = Column(Integer, primary_key=True)
    name = Column( String(200), nullable=False)
    timestamp = Column(DateTime, default=datetime.datetime.now)

Base.metadata.create_all(engine)

2021-11-09 17:33:10,858 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 17:33:10,859 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("table")
2021-11-09 17:33:10,860 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-09 17:33:10,862 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
from sqlalchemy.orm import mapper

class MyMapperTable(object):
    pass

mapper(MyMapperTable, table)

In [11]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

data1 = MySomeTable(name="SOMENAME")
data2 = MySomeTable(name="ANOTHERNAME")

session.add(data1)
session.add(data2)
print(session.new)
session.commit()

IdentitySet([<__main__.MySomeTable object at 0x7f0a98d06820>, <__main__.MySomeTable object at 0x7f0a98d06640>])
2021-11-09 17:33:22,867 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 17:33:22,869 INFO sqlalchemy.engine.Engine INSERT INTO "table" (name, timestamp) VALUES (?, ?)
2021-11-09 17:33:22,870 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ('SOMENAME', '2021-11-09 17:33:22.869248')
2021-11-09 17:33:22,871 INFO sqlalchemy.engine.Engine INSERT INTO "table" (name, timestamp) VALUES (?, ?)
2021-11-09 17:33:22,872 INFO sqlalchemy.engine.Engine [cached since 0.003084s ago] ('ANOTHERNAME', '2021-11-09 17:33:22.871635')
2021-11-09 17:33:22,873 INFO sqlalchemy.engine.Engine COMMIT
